<a href="https://colab.research.google.com/github/ssv273/Neural_Univesity/blob/main/hw_29.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import time
import tensorflow as tf
from tensorflow.keras.models import Model, load_model # из кераса подгружаем абстрактный класс базовой модели, метод загрузки предобученной модели
from tensorflow.keras.datasets import mnist #Библиотека с базой Mnist
from tensorflow.keras.models import Sequential # НС прямого распространения
from tensorflow.keras.layers import Dense, Embedding, LSTM, GRU, Input # из кераса загружаем необходимые слои для нейросети
from tensorflow.keras import utils # Утилиты для to_categorical
from tensorflow.keras.preprocessing import image # Для отрисовки изображения
from tensorflow.keras.optimizers import Adam, Adadelta # Алгоритмы оптимизации, для настройки скорости обучения
import numpy as np # Библиотека работы с массивами
import matplotlib.pyplot as plt # Отрисовка изображений

# Отрисовывать изображения в ноутбуке, а не в консоль или файл
%matplotlib inline


In [2]:
(x_train_org, y_train_org), (x_test_org, y_test_org) = mnist.load_data() #Загрузка данных Mnist

11501568/11490434 [==============================] - 0s 0us/step


Возьмем в качестве открывающего тега число 256, а в качестве закрывающего - 257

In [3]:
input_tensor = x_train_org.reshape(-1, 28*28)
input_tensor = input_tensor.astype(np.float32)
input_tensor = np.insert(input_tensor, 0, 256, axis=1)
input_tensor = np.insert(input_tensor, 785, 257, axis=1)
input_tensor.shape

(60000, 786)

In [4]:
target_tensor = utils.to_categorical(y_train_org, 10)
target_tensor = target_tensor.astype(np.float32)
target_tensor = np.insert(target_tensor, 0, 256, axis=1)
target_tensor = np.insert(target_tensor, 11, 257, axis=1)
target_tensor.shape

(60000, 12)

In [5]:
target_tensor = target_tensor[:1000]
input_tensor = input_tensor[:1000]

In [6]:
target_tensor[0]

array([256.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,
       257.], dtype=float32)

In [7]:
max_length_targ, max_length_inp = input_tensor.shape[1], target_tensor.shape[1]
max_length_targ, max_length_inp

(786, 12)

In [8]:
######################
# Создаем tf.data датасет (Раздел tf.data.Dataset API предлагает построить готовый конвейер для обучения моделей)
######################
# Метод .shuffle с параметром BUFFER_SIZE задаст случайность подачи тренировочных сэмплов в процессе обучения(против переобучения) 
BUFFER_SIZE = len(input_tensor) #укажем что случайно сэмплировать будем по всей длине обучающейся выборки
BATCH_SIZE = 16 #указываем размер батча
steps_per_epoch = len(input_tensor)//BATCH_SIZE # укажем количество шагов в одной эпохе
embedding_dim = 256 #размерность эмбеддинга, векторного пространства
units = 1024 #задаем размер слоя(количество нейронов в слое)

#!!!!!!!!!!!!!!!!!
vocab_inp_size = max_length_targ
vocab_tar_size = max_length_inp
#!!!!!!!!!!!!!!!!!


# Создаём датасет из массивов Numpy(рус и анг тренировочные фразы) со случайной подачей тренировочных сэмплов в процессе обучения
%time dataset = tf.data.Dataset.from_tensor_slices((input_tensor, target_tensor)).shuffle(BUFFER_SIZE)
# Передаем в датасет размер батча и указываем, что если в тренировке последний батч окажется неполным, то опустим его
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

# Посмотрим на форму примеров полученных батчей
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape
print(vocab_inp_size, vocab_tar_size)

CPU times: user 385 ms, sys: 316 ms, total: 702 ms
Wall time: 3.26 s
786 12


In [9]:




# vocab_inp_size = len(inp_lang_tokenizer.word_index)+1 #задаем размер немецкого словаря
# #vocab_tar_size = len(inp_lang_tokenizer.word_index)+1 #задаем размер английского словаря
# vocab_tar_size = len(targ_lang_tokenizer.word_index)+1 #задаем размер английского словаря



In [10]:
######################
# Создаем класс для кодировщика Encoder
######################
class Encoder(Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz): #указываем атрибуты класса
    super(Encoder, self).__init__() #даем возможность использовать и исполнять методы класса-родителя в классе потомке 
    self.batch_sz = batch_sz #атрибут возвращает размер батча
    self.enc_units = enc_units #атрибут возвращает размер слоя в кодировщике
    self.embedding = Embedding(vocab_size, embedding_dim) #атрибут эмбеддинга - слой Кераса с размером словаря на входе и с dim=256
    # Реккурентной сетью выберем GRU, указываем размер слоя, вывод из слоя в виде последовательностей, 
    # и метод инициализации весов 'glorot_uniform'(или метод Ксавьера) для упрощения прохождения сигнала при распростр-ии ошибки
    self.gru = GRU(self.enc_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')

  # Метод принимает входную фразу и начальное состояние
  def call(self, x, hidden): #при обращении к экземпляру класса как к функции, будет вызываться этот метод:
    x = self.embedding(x) # входящие тензоры преобразовываются в эмбеддинг
    output, state = self.gru(x, initial_state = hidden) #затем пропускаются через GRU и получаем выход + новое состояние
    return output, state #вызов метода/функции вернёт выход из сети GRU и состояние на выходе

  def initialize_hidden_state(self): #создаем метод инициализации состояний на скрытых слоях
    return tf.zeros((self.batch_sz, self.enc_units)) #задаем начальное состояние как размер батча на размер слоя
    
#  input tensor(фраза в виде индексов)---->Encoder----> output(значение на выходе) и hidden_state(состояние на выходе)

In [11]:
# Создадим модель кодировщика по уже заданным параметрам 
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# Подадим в качестве примера какой-то сэмпл(Тензор[64, 12]) на вход Encoder'у и визуализируем, что получим

sample_hidden = encoder.initialize_hidden_state() #инициализируем начальное скрытое состояние
# Даем Encoder'у сэмпл и начальное состояние, и получим выход из сети GRU и состояние на выходе (вызывается метод call класса Encoder)
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Размеры выхода из кодировщика: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Размеры скрытого состояния: (batch size, units) {}'.format(sample_hidden.shape))

Размеры выхода из кодировщика: (batch size, sequence length, units) (16, 786, 1024)
Размеры скрытого состояния: (batch size, units) (16, 1024)


In [12]:
######################
# Создаем класс Attention для внедрения механизма внимания
######################
class BahdanauAttention(Model): #название класса именем создателя механизма Дмитрия Богданова(Bahdanau)
  def __init__(self, units): # создаем слой внимания из указанного кол-ва нейронов
    super(BahdanauAttention, self).__init__() #даем возможность использовать и исполнять методы класса-родителя в классе потомке
    self.W1 = Dense(units) #атрибут: получаем веса, пропуская через полносвязный слой
    self.W2 = Dense(units) #атрибут: получаем веса, пропуская через полносвязный слой
    self.V =  Dense(1) #атрибут: пропускаем через Dense с одним нейроном, получаем отдельный вес на каждый такт

  def call(self, hidden_state, values): 
    # Форма состояния на скрытом слое (batch_size, hidden size)
    # Форму состояния на каждом такте увеличим до (batch_size, 1, hidden size)
    # Добавляем это для того, чтобы получить оценку
    hidden_with_time_axis = tf.expand_dims(hidden_state, 1)

    # Форма оценки score (размер батча, макс.длина слов на входе, 1), однёрка в конце, чтобы применить self.V
    # До применения self.V оценка была бы (размер батча, макс.длина слов на входе, количество нейронов в слое)
    score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))

    # К полученной оценке применим Софтмакс, который покажет вероятность полезности от 0 до 1 для каждого слова в фразе для декодера
    # Форма оценки score - (размер батча, макс.длина слов на входе, 1); Софтмакс применяем к оси "макс.длина слов"
    attention_weights = tf.nn.softmax(score, axis=1)

    # Построим вектор контекста 
    context_vector = attention_weights * values # веса внимания перемножим со значениями(выхода из кодировщика)
    # Сумму также применяем по оси "макс.длина слов на входе"
    context_vector = tf.reduce_sum(context_vector, axis=1) #размеры вектора контекста после суммирования будут (размер батча, размер слоя)

    return context_vector, attention_weights #возвращает вектор контекста и веса внимания

# encoder hidden state и  output ---->Attention Layer----> context vector(вектор контекста) и attention weights (веса внимания)

In [13]:
# Проверим, как работает слой
attention_layer = BahdanauAttention(10)
# Подадим на вход слою внимания выход из Encodera и его состояние, и получим значение и веса внимания
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Размеры значения внимания: (размер батча, размер слоя) {}".format(attention_result.shape))
print("Размеры весов внимания: (размер батча, длина последовательности, 1) {}".format(attention_weights.shape))

Размеры значения внимания: (размер батча, размер слоя) (16, 1024)
Размеры весов внимания: (размер батча, длина последовательности, 1) (16, 786, 1)


In [14]:
######################
# Создаем класс для декодировщика Decoder
######################
class Decoder(Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__() #даем возможность использовать и исполнять методы класса-родителя в классе потомке 
    self.batch_sz = batch_sz #атрибут возвращает размер батча
    self.dec_units = dec_units #атрибут возвращает размер слоя в декодере(кол-во нейронов)
    self.embedding = Embedding(vocab_size, embedding_dim) #атрибут эмбеддинга - слой Кераса с размером словаря на входе и (dim=256) на выходе

    # Реккурентной сетью выберем GRU, указываем размер слоя, вывод из слоя в виде последовательностей, 
    # и метод инициализации весов 'glorot_uniform'(или метод Ксавьера) для упрощения прохождения сигнала при распростр-ии ошибки    
    self.gru = GRU(self.dec_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
    self.fc = Dense(vocab_size) #атрибут вызовет полносвязный слой с размером словаря

    self.attention = BahdanauAttention(self.dec_units) #атрибут подключит механизм внимания, описанный ранее

  def call(self, x, hidden, enc_output):
    # enc_output размеры (batch_size, max_length, hidden_size - размер батча, макс.длина фраз, разм.скр.слоя)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # входящий тензор слова пропускаем через эмбеддинг (получаем размеры batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # дальше конкатенируем с вектором контекста (получаем размеры batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # сконкатенированный вектор передаем  в GRU и получаем выход с декодера и состояние
    output, state = self.gru(x)

    # output размеры (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # пропускаем через полносвязный слой
    x = self.fc(output) #output размеры (batch_size, vocab)

    return x, state, attention_weights

In [15]:
# Проверим работу декодера, подав на вход случайный массив с нужной размерностью
# Создали декодер с параметрами(размер анг.словаря, размерность эмбеддинга, кол-во нейронов, размер батча)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)
# Подаём на вход случайный массив с нужной размерностью, состояние и выход с кодировщика
sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)), sample_hidden, sample_output)

print ('Размер выхода с декодера: (размер батча, размер словаря) {}'.format(sample_decoder_output.shape))

Размер выхода с декодера: (размер батча, размер словаря) (16, 12)


In [16]:
######################
# Определяем функцию потерь
######################
optimizer = Adam() # из оптимизаторов кераса выбираем Adam
# Используем SparseCategoricalCrossentropy, к-я может работать не с категориальными лейблами
loss_object = tf.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none') #логит - тензор к к-му применим софтмакс

def loss_function(real, pred): # Запишем функцию потерь, на вход подаем фактический и предсказанный результат
  mask = tf.math.logical_not(tf.math.equal(real, 0)) #накидываем "маску" которая оставит для работы ненулевые значения
  loss_ = loss_object(real, pred) #фактические и предсказанные результаты передаем в SparseCategoricalCrossentropy и получаем ошибку

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask
  
  #reduce_mean - среднее любого выбранного тензора
  return tf.reduce_mean(loss_)

In [17]:
# Сохраняем процесс обучения модели чекпоинтами тензорфлоу
checkpoint_dir = './training_checkpoints' #даем ссылку на директорию
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt") #добавляем префикс "ckpt"
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder) #сохраняем состояния/показатели оптимизатора и моделей

In [18]:
######################
# Создадим функцию запуска обучения модели 
######################
@tf.function
def train_step(inp, targ, enc_hidden): #функция принимает тензоры(фразы в виде индексов) и состояние в кодировщике
  loss = 0 #создаем переменную, в которую будем записывать ошибку

  # Все операции по вычислению градиента записываются на ленту(tape) и мы получаем к ним доступ
  with tf.GradientTape() as tape:
    # Передаем тензор и начальное состояние в кодировщик и получим выход и состояние на выходе
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden #передадим это состояние декодеру

    # Передаем в качестве входа в декодер индекс токена "<start>"
    dec_input = tf.expand_dims([256] * BATCH_SIZE, 1)

    # Техника "Teacher forcing" - подаем предыдущее выходное слово на вход следущего в декодере. Targ.shape[64, 9]
    for t in range(1, targ.shape[1]): #для каждого слова из английской фразы
      # Передаем в обработку декодеру начальный токен, состояние на выходе из кодера, и выход из кодера
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output) # Получаем от декодера предсказание и обновленное состояние
      # Обновляем ошибку для текущих предсказаний
      loss += loss_function(targ[:, t], predictions)

      # Используем "Teacher forcing"
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1])) # Получаем ошибку на батче . Targ.shape[64, 9]. Делим на 9

  variables = encoder.trainable_variables + decoder.trainable_variables # создаем переменные, для которых TensorFlow будет вычислять градиенты

  gradients = tape.gradient(loss, variables) #отслеживаем градиент
  
  optimizer.apply_gradients(zip(gradients, variables)) #оптимизируем веса

  return batch_loss #функция обучения вернет ошибку на батче

In [19]:
######################
# Запуск обучения
######################
EPOCHS = 10 #устанавливаем количество эпох

In [20]:
for epoch in range(EPOCHS): #на каждой эпохе
  start = time.time() #включаем счетчик времени

  enc_hidden = encoder.initialize_hidden_state() #задаем начальное состояние на скрытом слое encodera
  total_loss = 0 #начальное значение итоговой ошибки

  # Для батча, входного и выходного тензора на каждом шаге эпохи
  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden) #передадим в функцию тензоры и состояние в кодировщике, обучим и получим ошибку на батче
    total_loss += batch_loss #добавим ее в итоговую ошибку
    # print(batch_loss)

  # Каждые 10 эпох будем сохранять чекпоинты
  if (epoch + 1) % 10 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)
  # Выведем показатели после каждой эпохи
  print('Эпоха {}/{} Ошибка {:.4f}'.format(epoch + 1, EPOCHS, total_loss / steps_per_epoch))
  print('Время на 1 эпоху {} сек\n'.format(round(time.time() - start), 1))

Эпоха 1/10 Ошибка nan
Время на 1 эпоху 34 сек

Эпоха 2/10 Ошибка nan
Время на 1 эпоху 21 сек

Эпоха 3/10 Ошибка nan
Время на 1 эпоху 21 сек

Эпоха 4/10 Ошибка nan
Время на 1 эпоху 21 сек

Эпоха 5/10 Ошибка nan
Время на 1 эпоху 21 сек

Эпоха 6/10 Ошибка nan
Время на 1 эпоху 21 сек

Эпоха 7/10 Ошибка nan
Время на 1 эпоху 21 сек

Эпоха 8/10 Ошибка nan
Время на 1 эпоху 21 сек

Эпоха 9/10 Ошибка nan
Время на 1 эпоху 21 сек

Эпоха 10/10 Ошибка nan
Время на 1 эпоху 21 сек



In [26]:
######################
# Функция запуска и оценки модели переводчика
######################
def evaluate(sentence):
    attention_plot = np.zeros((max_length_targ, max_length_inp)) # создаем начальные настройки графика внимания

    # sentence = preprocess_sentence(sentence) #предобрабатываем предложение
    sentence = tf.convert_to_tensor(sentence.reshape(1, 28*28+2))
    #inputs = [inp_lang_tokenizer.word_index[i] for i in sentence.split(' ')] #преобразовываем в послед-ть индексов
    # inputs = []
    # for i in sentence.split(' '):
    #   try:
    #     ind_inp = inp_lang_tokenizer.word_index[i]
    #     inputs.append(ind_inp)
    #   except:
    #     #ind_inp = " "
    #     print( f"В словаре нет слова '{i}'")
      

    # inputs = pad_sequences([inputs], maxlen=max_length_inp, padding='post') # делаем паддинг
    # inputs = tf.convert_to_tensor(inputs) #конвертируем в тф тензор

    result = [] #сюда запишем результат

    hidden = [tf.zeros((1, units))] #задаем начальное состояние
    enc_out, enc_hidden = encoder(sentence, hidden) #передаем его и входной тензор и получаем выход с кодера и состояние

    dec_hidden = enc_hidden #состояние кодера передаем в декодер
    dec_input = tf.expand_dims([1], 0) #передаем на вход декодеру <start> в виде индекса

    for t in range(max_length_targ): #идем по макс.длине фраз выходного языка(анг)
        # Прогоняем через декодер входящий тензор, состояние с выхода кодера, выход с кодера
        # Получаем результат предсказания, обновленное состояние, и веса внимания
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)

        # Сохраняем веса внимания для графика
        attention_weights = tf.reshape(attention_weights, (-1, ))
        # attention_plot[t] = attention_weights.numpy()
        # Аргмаксом вытаскиваем предсказанное слово
        predicted_id = tf.argmax(predictions[0]).numpy()
        # Результат конвертируем из индекса в слово и ложим в result = ''
        # result += targ_lang_tokenizer.index_word[predicted_id] + ' '
        result.append(predicted_id)
        # Если предсказанное слово - <end>, то останавливаемся, возвращаем результаты, выводим на графике
        # if targ_lang_tokenizer.index_word[predicted_id] == '<end>':
        #     return result, sentence, attention_plot

        # # предсказанное значение подается обратно в модель
        dec_input = tf.expand_dims([predicted_id], 0)

    return result

In [27]:
evaluate(input_tensor[0])

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
